## Housing Prices and Words

The data for this exercise has been scraped from Zillow and contains the price, the number of bedrooms, the number of bathrooms, the square footage, the location, and a description. The objective of this assignment is to use LASSO to understand which words contribute the most to price.

I have done most of the work for you in terms of processing the text. What is left is for you to fit the models at various levels of alpha and interpret the results.

In [104]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
import numpy as np 
from sklearn.linear_model import Lasso

data_dir = "C:/Users/jhtchns2/Box/ACE 592 SAE - Spring 2023/Class_Examples/4_Analysis/"
df = pd.read_csv(data_dir+'ZillowWebscrap_Champaign.csv')

In [105]:
sw_list = stopwords.words('english') + ['amp',''," "]

vct = CountVectorizer(stop_words=sw_list)
tfidf = TfidfVectorizer(stop_words=sw_list)

X = vct.fit_transform(df['description']).todense()
X = pd.DataFrame(X,columns=["word_" + x for x in vct.get_feature_names_out()])
df = pd.concat([df,X],axis=1) 

X_tfidf = tfidf.fit_transform(df['description']).todense()
X_tfidf = \
pd.DataFrame(X_tfidf,
             columns = ["word_tfidf_" +x for x in tfidf.get_feature_names_out()])
df = pd.concat([df,X_tfidf],axis=1) 

cols = ['beds', 'baths', 'sq_feet','lat','lon'] + list(df.columns[10:])

X = StandardScaler().fit_transform(df[cols])

y = StandardScaler().fit_transform(df[['price']])

def get_coefs(alpha,features,target):
    X = StandardScaler().fit_transform(df[features])
    y = StandardScaler().fit_transform(df[target])
    
    lasso_model =Lasso(alpha=alpha)
    lasso_model = lasso_model.fit(y=y,X=X)
    
    coefs = lasso_model.coef_
    
    return coefs

Alpha = np.arange(0,1,.01)